In [7]:
import numpy as np
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import tensorflow as tf
from transformers import BertTokenizer
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from keras.callbacks import Callback
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, GlobalAveragePooling1D, Dropout, LayerNormalization, MultiHeadAttention
from tensorflow.keras.layers import Layer, GRU, Bidirectional, Dense, Input, Reshape, GlobalAveragePooling1D
import nltk
from nltk.corpus import wordnet
import random
from lime.lime_text import LimeTextExplainer

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mhose\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
df = pd.read_csv(r"F:\MTL Cyberbullying\Cricket - Sheet1.csv")
df.head()

,Unnamed: 0,Unnamed: 1,Text,Category,Polarity
0,NaN,NaN,জয় বাংলা কাপ! তাও আবার স্বাধীনতার মাস মার্চে। ...,other,positive
1,NaN,NaN,জয় বাংলা কাপ! তাও আবার স্বাধীনতার মাস মার্চে। ...,team,positive
2,NaN,NaN,বাংলাদেশের পরে ভারতের সাপর্ট ই করি ?,team,positive
3,NaN,NaN,সৌম্যকে বাদ দেওয়া হোক,batting,negative
4,NaN,NaN,"প্রথমটি হচ্ছে, কোচ অত:পর সাকিব,সাকিব আর সাকিবর...",team,positive


In [9]:
df = df[['Text', 'Category', 'Polarity']]
df.head()

,Text,Category,Polarity
0,জয় বাংলা কাপ! তাও আবার স্বাধীনতার মাস মার্চে। ...,other,positive
1,জয় বাংলা কাপ! তাও আবার স্বাধীনতার মাস মার্চে। ...,team,positive
2,বাংলাদেশের পরে ভারতের সাপর্ট ই করি ?,team,positive
3,সৌম্যকে বাদ দেওয়া হোক,batting,negative
4,"প্রথমটি হচ্ছে, কোচ অত:পর সাকিব,সাকিব আর সাকিবর...",team,positive


In [10]:
df['Category'].value_counts()

Category
other              1010
team                731
batting             580
team management     329
bowling             329
Name: count, dtype: int64

In [11]:
df['Polarity'].value_counts()

Polarity
negative    2152
positive     566
neutral      261
Name: count, dtype: int64

In [12]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Initialize Bengali stopwords and lemmatizer
stop_words = set(stopwords.words('bengali'))
lemmatizer = WordNetLemmatizer()

# Function to clean text
def clean_text(text):
    text = re.sub(r'[^\u0980-\u09FF\s]', '', text)  # Keep only Bengali characters
    text = re.sub(r'\d+', '', text)                 # Remove numbers
    text = re.sub(r'\s+', ' ', text).strip()        # Remove extra spaces

    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    
    return ' '.join(words)

df['Text'] = df['Text'].apply(clean_text)
df.head()

,Text,Category,Polarity
0,জয় বাংলা কাপ স্বাধীনতার মাস মার্চে মাথা চমৎকার...,other,positive
1,জয় বাংলা কাপ স্বাধীনতার মাস মার্চে মাথা চমৎকার...,team,positive
2,বাংলাদেশের ভারতের সাপর্ট,team,positive
3,সৌম্যকে বাদ,batting,negative
4,প্রথমটি কোচ অতপর সাকিবসাকিব সাকিবরে দলে,team,positive


In [13]:
from transformers import BertTokenizer
import tensorflow as tf

# Load the pre-trained tokenizer
tokenizer = BertTokenizer.from_pretrained('sagorsarker/bangla-bert-base')

f:\Mini Conda\envs\env\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [14]:
# Verify that there are no more null values
print(df.isnull().sum())

Text        0
Category    0
Polarity    0
dtype: int64


In [15]:
# Define a function to tokenize a single sentence
def tokenize_data(df, max_length=128):
    input_ids = []
    attention_masks = []
    
    for sentence in df['Text']:
        encoded_dict = tokenizer.encode_plus(
            sentence,
            add_special_tokens=True, 
            max_length=max_length,    
            padding='max_length',    
            truncation=True,           
            return_attention_mask=True, 
            return_tensors='tf'        
        )
        
        # Append to lists
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
    
    # Convert lists to tensors
    input_ids = tf.convert_to_tensor(input_ids)
    attention_masks = tf.convert_to_tensor(attention_masks)
    
    # Squeeze the extra dimension
    input_ids = tf.squeeze(input_ids, axis=1)
    attention_masks = tf.squeeze(attention_masks, axis=1)
    
    return input_ids, attention_masks

input_ids, attention_masks = tokenize_data(df)

label_1 = tf.convert_to_tensor(df['Category'])
label_2 = tf.convert_to_tensor(df['Polarity'])

print(f"Input IDs shape: {input_ids.shape}")
print(f"Attention masks shape: {attention_masks.shape}")
print(f"Label 1 (Category) shape: {label_1.shape}")
print(f"Label 2 (Polarity) shape: {label_2.shape}")

Input IDs shape: (2979, 128)
Attention masks shape: (2979, 128)
Label 1 (Category) shape: (2979,)
Label 2 (Polarity) shape: (2979,)


In [16]:
# Ensure input_ids and attention_masks are converted to integer type tensors
input_ids = tf.convert_to_tensor(input_ids, dtype=tf.int32)
attention_masks = tf.convert_to_tensor(attention_masks, dtype=tf.int32)

In [20]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Initialize label encoders for the string labels
label_encoder_1 = LabelEncoder()
label_encoder_2 = LabelEncoder()

# Encode string labels into integers
df['Category'] = label_encoder_1.fit_transform(df['Category'])
df['Polarity'] = label_encoder_2.fit_transform(df['Polarity'])

# Convert labels to TensorFlow tensors
label_1 = tf.convert_to_tensor(df['Category'], dtype=tf.int32)
label_2 = tf.convert_to_tensor(df['Polarity'], dtype=tf.int32)

# Ensure input_ids and attention_masks are correctly formatted as tensors
input_ids = tf.convert_to_tensor(input_ids, dtype=tf.int32)
attention_masks = tf.convert_to_tensor(attention_masks, dtype=tf.int32)

# Split the data into training and testing sets
X_train, X_test, att_mask_train, att_mask_test, y_cat_train, y_cat_test, y_gender_train, y_gender_test = train_test_split(
    input_ids.numpy(), attention_masks.numpy(),
    label_1.numpy(), label_2.numpy(),  
    test_size=0.2, random_state=42
)

In [21]:
input_shape = X_train.shape[1]


In [22]:
import tensorflow as tf

def create_bilstm_model(input_shape):
    input_ids = tf.keras.layers.Input(shape=(input_shape,), dtype='int32', name='input_ids')
    attention_masks = tf.keras.layers.Input(shape=(input_shape,), dtype='int32', name='attention_masks')

    # Embedding layer
    embedding_layer = tf.keras.layers.Embedding(input_dim=tokenizer.vocab_size, output_dim=128)(input_ids)

    # First BiLSTM layer with dropout
    lstm_output = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(128, return_sequences=True, dropout=0.3))(embedding_layer)

    # Second BiLSTM layer
    lstm_output_2 = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(64, return_sequences=False, dropout=0.3))(lstm_output)

    # Dense layer before output layers
    dense_layer = tf.keras.layers.Dense(64, activation='relu')(lstm_output_2)

    # Dropout layer for regularization
    dropout_layer = tf.keras.layers.Dropout(0.3)(dense_layer)

    # Output layers for multi-task learning
    output_category = tf.keras.layers.Dense(5, activation='softmax', name='Category')(dropout_layer)
    output_polarity = tf.keras.layers.Dense(3, activation='softmax', name='Polarity')(dropout_layer)

    # Define the model with inputs and outputs
    model = tf.keras.Model(inputs=[input_ids, attention_masks],
                           outputs=[output_category, output_polarity])

    return model

# Instantiate the BiLSTM model
bilstm_model = create_bilstm_model(input_shape)

# Compile the model
bilstm_model.compile(
    optimizer='adam',
    loss={'Category': 'sparse_categorical_crossentropy', 'Polarity': 'sparse_categorical_crossentropy'},
    metrics={'Category': 'accuracy', 'Polarity': 'accuracy'}
)

# Display the model summary
bilstm_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 128, 128)     13052800    ['input_ids[0][0]']              
                                                                                                  
 bidirectional (Bidirectional)  (None, 128, 256)     263168      ['embedding[0][0]']              
                                                                                                  
 bidirectional_1 (Bidirectional  (None, 128)         164352      ['bidirectional[0][0]']          
 )                                                                                            

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
         
# Define EarlyStopping callback
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=3,       
    restore_best_weights=True 
)

# Train the model with EarlyStopping
history = bilstm_model.fit(
    [X_train, att_mask_train], 
    {'Category': y_cat_train, 'Polarity': y_gender_train}, 
    validation_split=0.1,
    epochs=10,
    batch_size=32,
    callbacks=[early_stopping]  
)


Epoch 1/10
67/67 [==============================] - 20s 80ms/step - loss: 2.3982 - Category_loss: 1.5661 - Polarity_loss: 0.8321 - Category_accuracy: 0.2957 - Polarity_accuracy: 0.7094 - val_loss: 2.2652 - val_Category_loss: 1.4758 - val_Polarity_loss: 0.7895 - val_Category_accuracy: 0.4351 - val_Polarity_accuracy: 0.7071
Epoch 2/10
67/67 [==============================] - 4s 58ms/step - loss: 2.1308 - Category_loss: 1.4037 - Polarity_loss: 0.7271 - Category_accuracy: 0.4118 - Polarity_accuracy: 0.7355 - val_loss: 1.9988 - val_Category_loss: 1.2871 - val_Polarity_loss: 0.7117 - val_Category_accuracy: 0.4854 - val_Polarity_accuracy: 0.7490
Epoch 3/10
67/67 [==============================] - 4s 58ms/step - loss: 1.7191 - Category_loss: 1.1083 - Polarity_loss: 0.6108 - Category_accuracy: 0.5429 - Polarity_accuracy: 0.7626 - val_loss: 2.0150 - val_Category_loss: 1.3259 - val_Polarity_loss: 0.6891 - val_Category_accuracy: 0.4603 - val_Polarity_accuracy: 0.7531
Epoch 4/10
67/67 [============

In [24]:
# Evaluate the model on the test set
results = bilstm_model.evaluate(
    [X_test, att_mask_test],
    {'Category': y_cat_test, 'Polarity': y_gender_test}
)

print(f"Test Loss and Accuracy: {results}")

19/19 [==============================] - 1s 36ms/step - loss: 2.0275 - Category_loss: 1.3242 - Polarity_loss: 0.7034 - Category_accuracy: 0.4513 - Polarity_accuracy: 0.7265
Test Loss and Accuracy: [2.027545928955078, 1.3241503238677979, 0.7033955454826355, 0.45134228467941284, 0.7265100479125977]


In [30]:
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
import numpy as np

# Step 1: Get Predictions
predictions = bilstm_model.predict([X_test, att_mask_test])

# Step 2: Convert predictions to class labels
y_cat_pred = np.argmax(predictions[0], axis=1)   # 'Category' prediction
y_gender_pred = np.argmax(predictions[1], axis=1)  # 'Polarity' prediction

# Step 3: Generate Classification Report with zero_division specified
# For Category
print("Classification Report for Category:")
print(classification_report(y_cat_test, y_cat_pred, zero_division=0))

# For Polarity
print("\nClassification Report for Polarity:")
print(classification_report(y_gender_test, y_gender_pred, zero_division=0))

# If you want the macro-averaged precision, recall, and F1 scores separately:
cat_precision = precision_score(y_cat_test, y_cat_pred, average='macro', zero_division=0)
cat_recall = recall_score(y_cat_test, y_cat_pred, average='macro', zero_division=0)
cat_f1 = f1_score(y_cat_test, y_cat_pred, average='macro', zero_division=0)

gender_precision = precision_score(y_gender_test, y_gender_pred, average='macro', zero_division=0)
gender_recall = recall_score(y_gender_test, y_gender_pred, average='macro', zero_division=0)
gender_f1 = f1_score(y_gender_test, y_gender_pred, average='macro', zero_division=0)

print("\nMacro-Averaged Scores for Category:")
print(f"Precision: {cat_precision:.4f}, Recall: {cat_recall:.4f}, F1 Score: {cat_f1:.4f}")

print("\nMacro-Averaged Scores for Polarity:")
print(f"Precision: {gender_precision:.4f}, Recall: {gender_recall:.4f}, F1 Score: {gender_f1:.4f}")


19/19 [==============================] - 1s 27ms/step
Classification Report for Category:
              precision    recall  f1-score   support

           0       0.37      0.56      0.45       117
           1       0.42      0.56      0.48        62
           2       0.51      0.76      0.61       209
           3       0.37      0.07      0.12       146
           4       0.00      0.00      0.00        62

    accuracy                           0.45       596
   macro avg       0.33      0.39      0.33       596
weighted avg       0.39      0.45      0.38       596


Classification Report for Polarity:
              precision    recall  f1-score   support

           0       0.79      0.88      0.83       433
           1       0.00      0.00      0.00        45
           2       0.46      0.43      0.45       118

    accuracy                           0.73       596
   macro avg       0.42      0.44      0.43       596
weighted avg       0.66      0.73      0.69       596


Ma